<left>FINM 33160 - Machine Learning In Finance</left>
<left>Winter 2023</left>
<br>
<h1><center> Homework 3 </center></h1>
<center> Due - 23:59 [CST] February 29th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

<h3> Imports </h3>

In [1]:
%matplotlib inline

In [2]:
import os
import pandas as pd
pd.set_option('use_inf_as_na', True)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
import pickle

import optuna
from optuna.trial import Trial
optuna.logging.set_verbosity(optuna.logging.FATAL)
from functools import partial
import warnings
warnings.filterwarnings("ignore")

<h3> Constants </h3>

In [3]:
home_dir = r'C:\Users\kwhyu\OneDrive - The University of Chicago\2023-1 Winter\FINM 33160\FINM-33160-W23'
raw_data_dir = r'dataset.pkl'
shap_features_dir = [r'Lecture 7\shap_features_ada.pkl',
                 r'Lecture 7\shap_features_ada_01.pkl',
                 r'Lecture 7\shap_features_ada_02.pkl']
training_dates = {'start': '2001-01-01', 'end': '2004-01-01'}
validation_dates = {'start': '2004-01-01', 'end': '2004-04-01'}
testing_dates = {'start': '2003-01-01', 'end': '2018-01-01'}

<h3> Helper-Functions </h3>

In [4]:
def f(x):

    if x > 0.01:
        return 1
    elif x < -0.025:

        return -1


    else:

        return 0

In [5]:
def max_drawdown(returns):
        local_max = [n for n in range(len(returns)-1) if ((n==0) and (returns[0] > returns[1])) or
           ((n > 0) and  (returns[n-1]<returns[n]) and (returns[n+1]<returns[n]))]

        local_min = [n for n in range(1,len(returns)) if ((n == len(returns)-1) and (returns[-1] < returns[-2])) or
                (returns[n-1]>returns[n]) and (returns[n+1]>returns[n])]

        def next_local_min(n):
            if [m for m in local_min if m > n]:
                return [m for m in local_min if m > n][0]
            else: return None

        drawdowns = [(n,next_local_min(n)) for n in local_max]
        drawdown_values = [returns[n] - returns[m] for (n,m) in drawdowns if m != None]
        if drawdown_values:
            return  np.max(drawdown_values)
        else: return 0.0

In [6]:
def accuracy_objective(trial:Trial,train=None,labels=None,val=None,val_labels=None):

    ada_m_depth = trial.suggest_int('max_depth',1,5)
    ada_n_estimators = trial.suggest_int('n_estimators', 10,100,step=10)
    ada_learning_rate = trial.suggest_float('learning_rate',0.1,0.5, step=0.1)


    ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=ada_m_depth),
                                 n_estimators=ada_n_estimators,
                                 learning_rate=ada_learning_rate,
                                 algorithm="SAMME.R")

    ada_clf.fit(train,labels)
    accuracy = ada_clf.score(val,val_labels)

    return accuracy

In [7]:
def mdd_objective(trial:Trial,train=None,labels=None,val=None,val_rets=None):

    ada_m_depth = trial.suggest_int('max_depth',1,5)
    ada_n_estimators = trial.suggest_int('n_estimators', 10,100,step=10)
    ada_learning_rate = trial.suggest_float('learning_rate',0.1,0.5, step=0.1)


    ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=ada_m_depth),
                                 n_estimators=ada_n_estimators,
                                 learning_rate=ada_learning_rate,
                                 algorithm="SAMME.R")
    ada_clf.fit(train,labels)
    pred = ada_clf.predict(val)
    profit = (pred * val_rets).sum()
    position = np.sum(np.abs(pred))
    x = (1/position) * profit

    return -x

<h3> Data </h3>

In [8]:
raw_data = pd.read_pickle(os.path.join(home_dir, raw_data_dir))

data = raw_data[raw_data['market_cap'] > 1000.0]
data = data.copy()
data.fillna(0.0,inplace=True)

In [9]:
data['rel_performance'] = data['pred_rel_return'].apply(f)
data.reset_index(inplace=True,)
data.set_index('date',inplace=True)

In [10]:
df_train = data.loc[training_dates.get('start'):training_dates.get('end')]
df_valid = data.loc[validation_dates.get('start'):validation_dates.get('end')]

In [11]:
train = df_train.reset_index().drop(['ticker','date',
                                     'next_period_return',
                                     'spy_next_period_return',
                                     'rel_performance','pred_rel_return',
                                     'return', 'cum_ret', 'spy_cum_ret'],axis=1)

valid = df_valid.reset_index().drop(['ticker','date',
                                    'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1)

In [12]:
valid_stock_returns = df_valid['next_period_return']

In [13]:
y_train = df_train['rel_performance'].values
y_valid = df_valid['rel_performance'].values

In [14]:
optimal_features_collection = []
for features_dir in shap_features_dir:
    with open(os.path.join(home_dir, features_dir), 'rb') as f:
        optimal_features_collection.append(pickle.load(f))

In [15]:
windows = len(optimal_features_collection[0])
start_dates = [pd.to_datetime(testing_dates.get('start')) + pd.DateOffset(months = 3 * i) for i in range(windows)]
end_dates = [d + pd.DateOffset(months = 36) for d in start_dates]

training_frames = [data.loc[d:d+pd.DateOffset(months = 36)] for d in start_dates]
test_frames = [data.loc[d + pd.DateOffset(months=6):d+pd.DateOffset(months = 9)] for d in end_dates]

training_data = [d.reset_index().drop
                                 (['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in training_frames]

test_data = [d.reset_index().drop(['ticker','date',
                                   'next_period_return',
                                   'spy_next_period_return',
                                   'rel_performance','pred_rel_return',
                                  'return', 'cum_ret', 'spy_cum_ret'],axis=1) for d in test_frames]

training_labels = [d['rel_performance'].values for d in training_frames]
test_labels = [d['rel_performance'].values for d in test_frames]

<h2> Q1 </h2>

<h3> Optimizing by Accuracy </h3>

In [16]:
study_accuracy = optuna.create_study(direction="maximize")

In [17]:
%%time
study_accuracy.optimize(partial(accuracy_objective, train=train, labels=y_train, val=valid, val_labels=y_valid),
                        n_trials=20,n_jobs=-1)

CPU times: total: 21min 37s
Wall time: 4min 7s


In [18]:
study_accuracy.best_params

{'max_depth': 4, 'n_estimators': 20, 'learning_rate': 0.4}

In [19]:
accuracy_best_params = study_accuracy.best_params
max_depth = accuracy_best_params.get('max_depth')
del accuracy_best_params['max_depth']
ada_clf_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=max_depth),**accuracy_best_params)

In [20]:
%%time
total_data = []
for optimal_features in optimal_features_collection:
    x = [1]
    for i in range(len(optimal_features)-2):
            ada_clf_1.fit(training_data[i][optimal_features[i].values],training_labels[i])
            pred_i = ada_clf_1.predict(test_data[i][optimal_features[i].values])

            profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
            positions = np.sum(np.abs(pred_i))
            x.append(x[i] + (x[i]/positions) * profit_i)
    total_data.append(x)

ValueError: Found array with 0 sample(s) (shape=(0, 60)) while a minimum of 1 is required by AdaBoostClassifier.

In [21]:
for i in range(len(total_data)):
    x = total_data[i]
    Sharpe = x.mean()/x.std()
    MDD = max_drawdown(x)
    print(">>> Accuracy Maximization Objective, Optimal Features Set", i+1, "Strategy:")
    print("    Sharpe Ratio:", Sharpe)
    print("    Maximum Draw Down:", MDD)

TypeError: 'int' object is not iterable

<h3> Optimizing by Max Drawdown </h3>

In [ ]:
study_mdd = optuna.create_study(direction="minimize")

In [ ]:
%%time
study_mdd.optimize(partial(mdd_objective, train=train, labels=y_train, val=valid, val_rets=valid_stock_returns),
                   n_trials=20,n_jobs=-1)

In [ ]:
study_mdd.best_params

In [ ]:
mdd_best_params = study_mdd.best_params
max_depth = mdd_best_params.get('max_depth')
del mdd_best_params['max_depth']
ada_clf_2 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=max_depth),**mdd_best_params)

In [ ]:
%%time
total_data = []
for optimal_features in optimal_features_collection:
    x = [1]

    for i in range(len(optimal_features)-2):

            ada_clf_2.fit(training_data[i][optimal_features[i].values],training_labels[i])
            pred_i = ada_clf_2.predict(test_data[i][optimal_features[i].values])

            profit_i = (pred_i * test_frames[i]['next_period_return']).sum()
            positions = np.sum(np.abs(pred_i))
            x.append(x[i] + (x[i]/positions) * profit_i)
    total_data.append(x)

In [ ]:
for i in len(total_data):
    x = total_data[i]
    Sharpe = x.mean()/x.std()
    MDD = max_drawdown(x)
    print(">>> Max Drawdown Minimization Objective, Optimal Features Set", i+1, "Strategy:")
    print("    Sharpe Ratio:", Sharpe)
    print("    Maximum Draw Down:", MDD)

The best strategy appears to be a combination of Max Drawdown Minimization Objective ('max_depth' = 5, 'n_estimators' =
10, 'learning_rate = 0.5) and the 2nd set in the optimal features provided.